In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_excel('/content/CSD - Resource Room.xlsx')
print(df.head())

                Campus Resource Code    Description  Capacity Lecture  \
0  Kuala Lumpur Campus          C209  C209 - [C209]     102.0       Y   
1  Kuala Lumpur Campus          C403  C403 - [C403]      50.0       Y   
2  Kuala Lumpur Campus          C405  C405 - [C405]     135.0       Y   
3  Kuala Lumpur Campus          C401  C401 - [C401]      40.0       Y   
4  Kuala Lumpur Campus          C407  C407 - [C407]     100.0       Y   

  Tutorial Lab Workshop Resource Status  
0        Y   N        Y          Active  
1        Y   N        N          Active  
2        Y   N        N          Active  
3        Y   N        N          Active  
4        Y   N        N          Active  


In [6]:
#캠퍼스, 워크숍, NNN 드랍 후 공백 N으로 채움, capacity N인 경우 20으로 채움
Resource_Room = pd.DataFrame(df)
Resource_Room = Resource_Room.drop(labels='Campus', axis=1)
Resource_Room = Resource_Room.drop(labels='Workshop', axis=1)
Resource_Room = Resource_Room[~Resource_Room.apply(lambda row: row.astype(str).str.contains('IMus', case=False, na=False).any(), axis=1)]
Resource_Room = Resource_Room[~(Resource_Room[['Lecture', 'Tutorial', 'Lab']].eq('N').all(axis=1))]
Resource_Room = Resource_Room.fillna('N')
Resource_Room['Capacity'] = Resource_Room['Capacity'].replace('N', '20')
Resource_Room = Resource_Room.drop(Resource_Room[Resource_Room['Capacity'] == 0].index)
Resource_Room

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status
0,C209,C209 - [C209],102.0,Y,Y,N,Active
1,C403,C403 - [C403],50.0,Y,Y,N,Active
2,C405,C405 - [C405],135.0,Y,Y,N,Active
3,C401,C401 - [C401],40.0,Y,Y,N,Active
4,C407,C407 - [C407],100.0,Y,Y,N,Active
...,...,...,...,...,...,...,...
235,C307 - GBS,KL Block C - Third Floor,100.0,Y,Y,N,Active
236,Civil Workshop,"Civil Workshop, Level 4 Block E",30.0,N,N,Y,Active
237,C480 - FETBE Block C,"Robotic Lab (Block C, C480)",30.0,Y,Y,N,Active
238,FASMeetingRoom1,FAS Meeting Room 1,10.0,Y,Y,N,Active


In [7]:
print(Resource_Room[Resource_Room['Capacity'] == 'N'])

Empty DataFrame
Columns: [Resource Code, Description, Capacity, Lecture, Tutorial, Lab, Resource Status]
Index: []


In [8]:
#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['Y', 'Y', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
General = Resource_Room[Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))]
General.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 5 to 130
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Resource Code    7 non-null      object
 1   Description      7 non-null      object
 2   Capacity         7 non-null      object
 3   Lecture          7 non-null      object
 4   Tutorial         7 non-null      object
 5   Lab              7 non-null      object
 6   Resource Status  7 non-null      object
dtypes: object(7)
memory usage: 448.0+ bytes


In [9]:
#Gerneral하지 않은 경우를 뺀 나머지 데이터프레임 생성
conditions_to_exclude = [
    ['Y', 'Y', 'N'],
    ['N', 'Y', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']

#G_Resource_Room = Resource_Room[~Resource_Room[columns_to_check].apply(lambda row: tuple(row) in conditions_to_exclude, axis=1)]
#G_Resource_Room.info()

etc_Room = Resource_Room[~Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))]
etc_Room.info()



<class 'pandas.core.frame.DataFrame'>
Index: 178 entries, 0 to 240
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Resource Code    178 non-null    object
 1   Description      178 non-null    object
 2   Capacity         178 non-null    object
 3   Lecture          178 non-null    object
 4   Tutorial         178 non-null    object
 5   Lab              178 non-null    object
 6   Resource Status  178 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [10]:
#G_Resource_Room_IMus = G_Resource_Room[G_Resource_Room['Description'].str.contains('IMus', na=False) | G_Resource_Room['Resource Code'].str.contains('IMus', na=False)]
Kitchen = etc_Room[etc_Room.apply(lambda row: row.astype(str).str.contains('Kitchen', regex=False).any(), axis=1)]
Design = etc_Room[etc_Room['Description'].str.contains('DESIGN', case=False, na=False)]
df_combined = pd.concat([Kitchen, Design])
LL = etc_Room[~etc_Room.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]

In [11]:
#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['N', 'N', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
Lab = LL[LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions)) & ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]
Lab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 9 to 236
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Resource Code    40 non-null     object
 1   Description      40 non-null     object
 2   Capacity         40 non-null     object
 3   Lecture          40 non-null     object
 4   Tutorial         40 non-null     object
 5   Lab              40 non-null     object
 6   Resource Status  40 non-null     object
dtypes: object(7)
memory usage: 2.5+ KB


In [12]:
#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['Y', 'N', 'N']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
Lecture = LL[LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions)) & ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]
Lecture


,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status
123,Group Therapy Room,Group Therapy Room,15.0,Y,N,N,Active


In [13]:
df_combined = pd.concat([Lab, Lecture])
ETC = LL[~LL.apply(tuple, 1).isin(df_combined.apply(tuple, 1))]
ETC

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status
0,C209,C209 - [C209],102.0,Y,Y,N,Active
1,C403,C403 - [C403],50.0,Y,Y,N,Active
2,C405,C405 - [C405],135.0,Y,Y,N,Active
3,C401,C401 - [C401],40.0,Y,Y,N,Active
4,C407,C407 - [C407],100.0,Y,Y,N,Active
...,...,...,...,...,...,...,...
227,C219b,C219 - [C219b],25.0,Y,Y,N,Active
235,C307 - GBS,KL Block C - Third Floor,100.0,Y,Y,N,Active
237,C480 - FETBE Block C,"Robotic Lab (Block C, C480)",30.0,Y,Y,N,Active
238,FASMeetingRoom1,FAS Meeting Room 1,10.0,Y,Y,N,Active


In [14]:
import pandas as pd

General['Type'] = 'General'
Lab['Type'] = 'Lab'
Kitchen['Type'] = 'Kitchen'
Design['Type'] = 'Art'
Lecture['Type'] = 'Lecture'
ETC['Type'] = 'Etc'

df_final = pd.concat([General, ETC, Kitchen, Design, Lab, Lecture], ignore_index=True)
df_final = df_final.drop_duplicates(keep='first')


# 엑셀 파일로 저장 (index 제거)
df_final.to_excel("data_tq.xlsx", index=False)

print("엑셀 파일 저장 완료!")



엑셀 파일 저장 완료!


<ipython-input-14-b14b92c2eada>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  General['Type'] = 'General'
<ipython-input-14-b14b92c2eada>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lab['Type'] = 'Lab'
<ipython-input-14-b14b92c2eada>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-